In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# === Caminhos ===
RESULTADOS_DIR = Path("Resultados/")
RESULTADOS_DIR.mkdir(exist_ok=True)

In [3]:
def criar_features():
    print("⚙️ Carregando base unificada Netflix + IMDB...\n")

    # 1️⃣ Carregar dataset consolidado
    df = pd.read_parquet(RESULTADOS_DIR / "Netflix_IMDB_Matched.parquet", engine="fastparquet")

    print(f"📂 Registros carregados: {len(df):,}")
    print(f"📊 Colunas originais: {list(df.columns)}\n")

    # 2️⃣ Selecionar colunas relevantes
    colunas_relevantes = [
        "netflix_title", "imdb_title", "match_score", "titleType", "startYear",
        "runtimeMinutes", "averageRating", "numVotes", "genres", "assistido"
    ]
    df = df[colunas_relevantes].copy()

    # 3️⃣ Tratar nulos e tipos
    df["startYear"] = pd.to_numeric(df["startYear"], errors="coerce").fillna(-1).astype(int)
    df["runtimeMinutes"] = pd.to_numeric(df["runtimeMinutes"], errors="coerce").fillna(-1)
    df["averageRating"] = pd.to_numeric(df["averageRating"], errors="coerce").fillna(-1)
    df["numVotes"] = pd.to_numeric(df["numVotes"], errors="coerce").fillna(0)
    df["genres"] = df["genres"].fillna("Unknown")

    # 4️⃣ Filtrar apenas títulos com nota válida
    df = df[df["averageRating"] != -1].copy()
    print(f"🎬 Após filtro: {len(df):,} títulos com nota válida (≠ -1)\n")

    # 5️⃣ Criar feature de idade do título
    df["idade_titulo"] = 2025 - df["startYear"]
    df["idade_titulo"] = df["idade_titulo"].where(df["idade_titulo"] >= 0, 0)

    # 6️⃣ Expande gêneros e cria dummies com drop_first=True (k-1)
    df["genres"] = df["genres"].apply(lambda x: x if isinstance(x, list) else str(x).split(","))
    df_exploded = df.explode("genres")
    df_exploded["genres"] = df_exploded["genres"].str.strip()

    generos_dummies = pd.get_dummies(df_exploded["genres"], prefix="genre", drop_first=True)
    df = pd.concat([df, generos_dummies.groupby(df_exploded.index).max()], axis=1)

    # 7️⃣ Dummies para tipo de título (k-1)
    df = pd.get_dummies(df, columns=["titleType"], prefix="type", drop_first=True)

    # 8️⃣ Normalizar match_score (0–1)
    df["match_score"] = df["match_score"] / 100

    # 9️⃣ Reordenar colunas (assistido no fim)
    df = df[[c for c in df.columns if c != "assistido"] + ["assistido"]]

    # 🔟 Salvar dataset final
    df.to_parquet(RESULTADOS_DIR / "Base_Modelagem.parquet", index=False, engine="fastparquet")

    print("✅ Engenharia de features concluída com k-1 dummies e apenas notas válidas!")
    print("💾 Base final salva em 'Resultados/Base_Modelagem.parquet'")
    print(f"🔢 Total de features: {df.shape[1] - 1}")

    return df


# Executar
df_features = criar_features()
df_features.head(10)

⚙️ Carregando base unificada Netflix + IMDB...

📂 Registros carregados: 688
📊 Colunas originais: ['netflix_title', 'imdb_title', 'match_score', 'match_tipo', 'tconst', 'titleType', 'primaryTitle', 'originalTitle', 'startYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'titulo_limpo', 'assistido']

🎬 Após filtro: 654 títulos com nota válida (≠ -1)

✅ Engenharia de features concluída com k-1 dummies e apenas notas válidas!
💾 Base final salva em 'Resultados/Base_Modelagem.parquet'
🔢 Total de features: 36


,netflix_title,imdb_title,match_score,startYear,runtimeMinutes,averageRating,numVotes,genres,idade_titulo,genre_Adventure,...,genre_Sci-Fi,genre_Short,genre_Sport,genre_Talk-Show,genre_Thriller,genre_Unknown,genre_War,type_tvMovie,type_tvSeries,assistido
0,doom at your service,doom at your service,1.000000,2021,65.0,7.8,10596.0,"[Drama, Fantasy, Romance]",4,False,...,False,False,False,False,False,False,False,False,True,1
1,brooklyn ninenine,brooklyn ninenine,1.000000,2013,22.0,8.4,403619.0,"[Comedy, Crime]",12,False,...,False,False,False,False,False,False,False,False,True,1
2,love next door,love next door,1.000000,2024,60.0,7.6,9330.0,"[Comedy, Romance]",1,False,...,False,False,False,False,False,False,False,False,True,1
3,romantics anonymous,romantics anonymous,1.000000,2010,80.0,6.9,12410.0,"[Comedy, Romance]",15,False,...,False,False,False,False,False,False,False,False,False,1
4,beyond the bar,beyond the bar,1.000000,2025,60.0,7.9,3752.0,[Drama],0,False,...,False,False,False,False,False,False,False,False,True,1
5,because this is my first life,because this is my first life,1.000000,2017,70.0,8.0,10079.0,"[Comedy, Drama, Romance]",8,False,...,False,False,False,False,False,False,False,False,True,1
6,our beloved summer,our beloved summer,1.000000,2021,70.0,8.2,13620.0,"[Drama, Romance]",4,False,...,False,False,False,False,False,False,False,False,True,1
7,fight for my way,fight for my way,1.000000,2017,70.0,8.1,11790.0,"[Comedy, Romance]",8,False,...,False,False,False,False,False,False,False,False,True,1
8,alice in borderland,alice in borderland,1.000000,2020,50.0,7.8,135325.0,"[Action, Drama, Horror]",5,False,...,False,False,False,False,False,False,False,False,True,1
9,weightlifting fairy kim bok joo,weightlifting fairy kim bokjoo,0.885246,2016,60.0,8.3,15896.0,"[Comedy, Drama, Romance]",9,False,...,False,False,False,False,False,False,False,False,True,1
